# HARI PRASATH S

# Lab 12. Image corpus creation and Transfer Learning in CNN

##### 1. Import the necessary libraries

In [1]:
import datetime
import keras
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

##### 2. Initialize some parameters

In [2]:
now = datetime.datetime.now  #get current time

batch_size = 128
num_classes = 5
epochs = 5

img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

##### 3. Partition MINST dataset

In [3]:
#data, shuffled and split between train and test sets

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
#create 2 datasets: one with digits below 5 and one with 5 and above

x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [5]:
x_train_lt5.shape

(30596, 28, 28)

In [6]:
x_train_gte5.shape

(29404, 28, 28)

##### 4. Define the "feature" layers

In [7]:
feature_layers =[ 
    Conv2D(filters, kernel_size = (3,3), activation = 'relu',padding='valid',input_shape = (img_rows,img_cols, 1)),
    Conv2D(filters, kernel_size = (3,3), activation = 'relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),
    Flatten() 
]

##### 5. Define the "classification" layers

In [8]:
classification_layers = [
    Dense(128, activation = 'relu'),
    Dropout(0.5),
    Dense(num_classes, activation = 'softmax')
]

##### 6. Define a Sequential model

In [9]:
model = Sequential(feature_layers+classification_layers)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 128)               589952    
                                                        

##### 7. Create a function train_model()

In [10]:
def train_model(model,train,test,num_classes):
    train[0] = train[0].reshape(train[0].shape[0],img_rows,img_cols,1)
    test[0] = test[0].reshape(test[0].shape[0],img_rows,img_cols,1)

    train[0] = train[0].astype('float32')
    test[0] = test[0].astype('float32')

    train[0] /= 255
    test[0] /= 255

    print(train[0].shape)
    print(test[0].shape)
 
    s = pd.Series(train[1])
    train[1] = pd.get_dummies(s)
    train[1] = train[1].values

    s = pd.Series(test[1])
    test[1] = pd.get_dummies(s)
    test[1] = test[1].values

    model.compile(optimizer='adadelta',loss='categorical_crossentropy',metrics=['accuracy'])

    train_start_time=now()

    model.fit(train[0],train[1],epochs=epochs,verbose=2)
  
    print('\n')
    print('\n')

    print('Training time: %s' % (now() - train_start_time))

    score=model.evaluate(test[0],test[1],verbose=0)

    print('test loss ',score[0])
    print('test accuracy ',score[1])

##### 8. Training on digits 5 to 9

In [11]:
train_model(model, [x_train_gte5, y_train_gte5], [x_test_gte5, y_test_gte5], num_classes)


(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 26s - loss: 1.5849 - accuracy: 0.2688 - 26s/epoch - 28ms/step
Epoch 2/5
919/919 - 24s - loss: 1.5191 - accuracy: 0.4162 - 24s/epoch - 26ms/step
Epoch 3/5
919/919 - 25s - loss: 1.4363 - accuracy: 0.5346 - 25s/epoch - 27ms/step
Epoch 4/5
919/919 - 24s - loss: 1.3302 - accuracy: 0.6198 - 24s/epoch - 26ms/step
Epoch 5/5
919/919 - 23s - loss: 1.2066 - accuracy: 0.6776 - 23s/epoch - 25ms/step




Training time: 0:02:01.499758
test loss  1.081207513809204
test accuracy  0.8294589519500732


##### 9. Freeze Feature Layers

In [12]:
#Freeze only the feature layers

for l in feature_layers:
    l.trainable = False

##### 10. Print Summary

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 128)               589952    
                                                        

##### 11. Training for digits 0 to 4 based on digits 5 to 9

In [14]:
train_model(model, [x_train_lt5, y_train_lt5], [x_test_lt5, y_test_lt5], num_classes)


(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 14s - loss: 1.4970 - accuracy: 0.3767 - 14s/epoch - 14ms/step
Epoch 2/5
957/957 - 12s - loss: 1.3208 - accuracy: 0.5379 - 12s/epoch - 13ms/step
Epoch 3/5
957/957 - 12s - loss: 1.1759 - accuracy: 0.6686 - 12s/epoch - 12ms/step
Epoch 4/5
957/957 - 12s - loss: 1.0477 - accuracy: 0.7524 - 12s/epoch - 12ms/step
Epoch 5/5
957/957 - 12s - loss: 0.9338 - accuracy: 0.8006 - 12s/epoch - 13ms/step




Training time: 0:01:01.698501
test loss  0.817685604095459
test accuracy  0.8925861120223999


##### 12. Reversing the training process

In [15]:
modelR = Sequential(feature_layers+classification_layers)
modelR.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 128)               589952    
                                                      

In [16]:
train_model(modelR, [x_train_lt5, y_train_lt5], [x_test_lt5, y_test_lt5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 14s - loss: 0.8365 - accuracy: 0.8328 - 14s/epoch - 15ms/step
Epoch 2/5
957/957 - 14s - loss: 0.7597 - accuracy: 0.8529 - 14s/epoch - 15ms/step
Epoch 3/5
957/957 - 12s - loss: 0.6927 - accuracy: 0.8670 - 12s/epoch - 13ms/step
Epoch 4/5
957/957 - 12s - loss: 0.6350 - accuracy: 0.8762 - 12s/epoch - 12ms/step
Epoch 5/5
957/957 - 12s - loss: 0.5905 - accuracy: 0.8830 - 12s/epoch - 13ms/step




Training time: 0:01:04.031238
test loss  0.4846039414405823
test accuracy  0.9350067973136902


In [17]:
#Freeze only the feature layers

for l in feature_layers:
    l.trainable = False

In [18]:
modelR.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 128)               589952    
                                                      